In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.chdir('..')

In [3]:
from collections import defaultdict
from sklearn.model_selection import train_test_split

from fgclassifier.utils import read_data, get_dataset

X_train, y_train = read_data(get_dataset('train_en'), flavor=None)
X_test, y_test = read_data(get_dataset('valid_en'), flavor=None)

2018-12-03 21:02:07,120 [INFO] Reading data/english_train.csv..
2018-12-03 21:02:07,384 [INFO] Reading data/english_valid.csv..


In [4]:
fm = defaultdict(dict)

In [ ]:
from fgclassifier.features import FeaturePipeline, logger

for name in ['word2vec_en']:
    logger.info(f'Building features for {name}...')
    model = FeaturePipeline.from_spec(name, cache=fm)
    model.fit_transform(X_train)
    model.transform(X_test)

C:\Users\jyyjc\Anaconda3\envs\idp\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-12-03 21:02:13,350 [INFO] 'pattern' package not found; tag filters are not available for English
2018-12-03 21:02:13,383 [INFO] Building features for word2vec_en...
2018-12-03 21:02:37,467 [INFO] collecting all words and their counts
2018-12-03 21:02:37,469 [INFO] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-03 21:02:38,081 [INFO] collected 23515 word types from a corpus of 1877982 raw words and 8000 sentences
2018-12-03 21:02:38,083 [INFO] Loading a fresh vocabulary
2018-12-03 21:02:38,192 [INFO] effective_min_count=3 retains 10128 unique words (43% of original 23515, drops 13387)
2018-12-03 21:02:38,193 [INFO] effective_min_count=3 leaves 1861693 word corpus (99% of original 1877982, drops 16289)
2018-12-03 21:02:38,257 [INFO] deleting th

2018-12-03 21:03:14,532 [INFO] worker thread finished; awaiting finish of 0 more threads
2018-12-03 21:03:14,534 [INFO] EPOCH - 9 : training on 1877982 raw words (1291246 effective words) took 3.7s, 351252 effective words/s
2018-12-03 21:03:15,568 [INFO] EPOCH 10 - PROGRESS: at 27.23% examples, 342173 words/s, in_qsize 6, out_qsize 1
2018-12-03 21:03:16,570 [INFO] EPOCH 10 - PROGRESS: at 57.75% examples, 366636 words/s, in_qsize 6, out_qsize 0
2018-12-03 21:03:17,570 [INFO] EPOCH 10 - PROGRESS: at 84.83% examples, 359629 words/s, in_qsize 6, out_qsize 0


In [15]:
scores_train, avg_score_train = defaultdict(dict), defaultdict(dict)
scores_test, avg_score_test = defaultdict(dict), defaultdict(dict)

In [16]:
from fgclassifier.train import fm_cross_check
from IPython.display import clear_output

In [17]:
conf = {
    'fm_cache': fm,
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test,
    'results': {
        'models': {},
        'avg': avg_score_test,
        'all': scores_test,
        'avg_train': avg_score_train,
        'all_train': scores_train,
    }
}

# All other models can run on many classifiers
results = fm_cross_check(
    ['word2vec'],
    ['RBF', 'LDA', 'QDA', 'LinearSVC', 'Ridge', 'Logistic'], **conf)

clear_output()

2018-12-03 21:01:49,839 [INFO] 
2018-12-03 21:01:49,841 [INFO] ============ Feature Model: word2vec ============
2018-12-03 21:01:49,843 [INFO] 
2018-12-03 21:01:49,845 [INFO] Train for word2vec -> RBF...


KeyError: 'model'

In [ ]:
rows = {}
for fm_name in all_scores:
    for clf_name in all_scores[fm_name]:
        key = f'{fm_name}.{clf_name}'
        rows[key] = [all_avg_scores[fm_name][clf_name],
                     *all_scores[fm_name][clf_name]]
df = pd.DataFrame(rows)
df.index = ['average', *y_train.columns]
df = df.T.sort_values('average', ascending=False)
df

In [10]:
import numpy as np

print(np.mean(scores))

0.40654159090857
